In [ ]:
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import uuid
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# ✅ Assign the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# ✅ Use it here
openai_client = OpenAI(
    api_key=openai_api_key
)

In [36]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from bs4 import BeautifulSoup
import pandas as pd
import uuid
from openai import OpenAI # ✅ add openai import


# Utility functions
def clean(text):
    if pd.isna(text): return ""
    return str(text).strip()

def strip_html(text):
    return BeautifulSoup(text, "html.parser").get_text(separator=" ", strip=True)



def get_openai_embedding(text: str) -> list:
    response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# Load CSV
df = pd.read_csv("/Users/diegohernandez/Downloads/Maestri Milano - Products.csv")
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

# Connect to Qdrant
client = QdrantClient(host="vps.maestri.com.co", port=6333, https=False)
collection_name = "maestri_products"
embedding_size = 1536  # ✅ OpenAI embedding size is 1536

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embedding_size, distance=Distance.COSINE),
)

points = []

for _, row in df.iterrows():
    if pd.isna(row.get("published_on")):
        continue

    product_name = clean(row.get("nombre"))
    bodega = clean(row.get("bodega"))
    region = clean(row.get("región"))
    tipo = clean(row.get("tipo"))
    maridaje1 = clean(row.get("maridaje_1"))
    maridaje2 = clean(row.get("maridaje_2"))
    maridaje = " y ".join([m for m in [maridaje1, maridaje2] if m])
    notas = clean(row.get("notas_de_cata"))
    descripcion = strip_html(clean(row.get("descripción")))
    precio = clean(row.get("precio"))
    category = clean(row.get("pasillo"))
    alternate_names = ""  # future improvements if needed

    short_text = f"""\
Producto: {product_name}. Tipo: {tipo}. Bodega: {bodega}. Región: {region}.
Maridaje: {maridaje}. Notas: {notas}. Descripción: {descripcion}.
También conocido como: {alternate_names}. Precio: {precio}.
"""

    if not short_text.strip():
        continue

    # ✅ Now using OpenAI to generate the embedding
    vector = get_openai_embedding(short_text)

    payload = {
        "product_name": product_name,
        "bodega": bodega,
        "region": region,
        "tipo": tipo,
        "precio": precio,
        "notas": notas,
        "descripcion": descripcion,
        "maridaje": maridaje,
        "category": category,
        "alternate_names": alternate_names
    }

    points.append(PointStruct(id=str(uuid.uuid4()), vector=vector, payload=payload))

# Insert into Qdrant
client.upsert(collection_name=collection_name, points=points)
print(f"✅ Inserted {len(points)} products into Qdrant collection: {collection_name}")


/var/folders/xq/gclpc2_d3ql92h6tddvfwkjr0000gn/T/ipykernel_27573/164065924.py:35: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


KeyboardInterrupt: 

In [3]:
df.columns

Index(['nombre', 'slug', 'collection_id', 'locale_id', 'item_id', 'created_on',
       'updated_on', 'published_on', 'precio', 'imagen_del_producto',
       'categoria', 'descripción', 'maridaje_1', 'maridaje_2', 'notas_de_cata',
       'temperatura_de_servicio', 'pasillo', 'tipo', 'región',
       'cepa_principal', 'ocasión', 'bodega', 'denominación', 'peso_/_volumen',
       'gr/ml', 'item_id', 'precio_descuento', 'categories', 'precios',
       'descuento', 'descuento_2x1', 'descuento_3x2', 'ciudad',
       'productoreserva', 'descuento%off'],
      dtype='object')

In [5]:
# Step 1 - Inspect products with "prosecco" in product_name
print("\n🔍 Checking indexed Prosecco products by name:\n")

results, _ = client.scroll(
    collection_name=collection_name,
    limit=100,
    with_payload=True
)

prosecco_items = [
    r for r in results if "prosecco" in str(r.payload.get("product_name", "")).lower()
]

if not prosecco_items:
    print("❌ No Prosecco products found in product_name.")
else:
    for i, item in enumerate(prosecco_items, 1):
        print(f"\n#{i}")
        for k, v in item.payload.items():
            print(f"{k}: {v}")



🔍 Checking indexed Prosecco products by name:


#1
product_name: Prosecco Sior Piero
bodega: V8+
region: Veneto
tipo: Espumoso
precio: 115000.0
notas: De un color pajizo brillante, burbujas finas y persistentes, aromas delicados que recuerdan notas Florales y frutales que recuerdan a la manzanilla, durazno, peras. En el paladar es fresco, elegante con una pequeña cantidad de azúcar residual que es balanceado por la acidez.
descripcion: ELABORACIÓN DEL VINO   Después de la vendimia manual, la uva es separada de los raspones y son estrujadas suavemente. La fermentación toma lugar en tanques de acero inoxidable, a temperatura controlada para preservar los aromas. La segunda fermentación se lleva a cabo en autoclaves de acero por aproximadamente 35 días. VARIEDAD DE UVA Y ÁREA DE PRODUCCIÓN   Glera 100%. Los viñedos se encuentran en las colinas de Valdobbiadene, la patria del Prosecco, que son las burbujas más famosas de Italia. Aromas inconfundibles de este estilo de vino. La región es p